## Revision History

In [ ]:
# Change_date         revision_number     change_description                           author
# 06/03/2024          1                   initial check-in                             Kranthi
# 


In [ ]:
import concurrent.futures
from delta import *
from pyspark.sql.types import StructType, StructField, ArrayType, StringType, LongType, DoubleType, BooleanType, MapType,IntegerType
from pyspark.sql.functions import *
from functools import reduce
from pyspark.sql.dataframe import DataFrame
import pyspark.sql.functions as F
import json
import base64
from datetime import datetime,timedelta
from time import sleep
spark.conf.set("spark.sql.sources.partitionOverwriteMode","DYNAMIC")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

In [ ]:
%run /utils/common_functions

## define a dict for all brands

In [ ]:
property_brand  = {309713727:'Bates US'
,309561205:'CatFootwear Canada'
,309621431:'CatFootwear DE'
,309599233:'CatFootwear EMEAEmerging'
,309613069:'CatFootwear UK'
,309592771:'CatFootwear US'
,309589213:'Chacos US'
,309616895:'HushPuppies Canada'
,309650146:'HushPuppies US'
,309619975:'Hytest US'
,309607124:'Keds Canada'
,309606225:'Merrell BE'
,309607810:'Merrell Canada'
,309615440:'Merrell DE'
,309628914:'Merrell EMEAEmerging'
,309620743:'Merrell ES'
,309579645:'Merrell FR'
,309621834:'Merrell NL'
,309591477:'Merrell SE'
,309599329:'Merrell UK'
,309592118:'OnlineShoes US'
,310709711:'Saucony AT'
,309600562:'Saucony BE'
,309596198:'Saucony Canada'
,309624452:'Saucony DE'
,309643444:'Saucony EMEAEmerging'
,309629089:'Saucony ES'
,309607948:'Saucony FR'
,309537329:'Saucony IT'
,309587387:'Saucony UK'
,309650574:'Sperry Canada'
,309599656:'Wolverine Canada'
}
#,309617502:'Merrell US'}
#,309591384:'Wolverine US', ,309603632:'Saucony NL'
# property_brand  = {
#  309591384:'Wolverine US'}

#property_brand  = {309603632:'Saucony NL'}
print(property_brand)

In [ ]:
project = "ga360-connection-267115"

token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary  
#token_library.getSecret("kv-name", "secret-name", "linked-service")  
ga4_credentials = token_library.getSecret(kv_name, "GA4-credentials", "ls_kv_adap")  
print(ga4_credentials)

In [ ]:

spark.conf.set("viewsEnabled","true")
#spark.conf.set("materializationDataset","ga360-connection-267115.analytics_297479542.events_20240201")
spark.conf.set("materializationDataset","analytics_297479542")
def get_property_partitions(p_property_id,p_tab_type): 
    print('get partitions for property::',p_property_id)
    property_date_tup_list = []
    if p_tab_type == 'events':
        query = f"""select table_name,'events' as tab_type
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_events_2023%'""" 
    elif p_tab_type == 'transactions':
        query = f"""select table_name,'transactions' as tab_type
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_transactions_2023%' """
    elif p_tab_type == 'acquisition_sessions':
         query = f"""select table_name,'acquisition_sessions' as tab_type
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_acquisition_sessions_2023%'"""  
    elif p_tab_type == 'transaction_qty':
         query = f"""select table_name,'transaction_qty' as tab_type
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_transaction_qty_2023%'"""
    elif p_tab_type == 'all':
        query = f"""select table_name,'events' as tab_type
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_events_2023%'            
             UNION ALL
             select table_name,'transactions'
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_transactions_2023%'             
             UNION ALL
             select table_name,'acquisition_sessions'
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_acquisition_sessions_2023%'            
             UNION ALL
             select table_name,'transaction_qty'
             from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
             where table_schema = 'analytics_{p_property_id}' 
             and table_name like 'reported_transaction_qty_2023%'"""
        # query = f"""select table_name,'events' as tab_type
        #      from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
        #      where table_schema = 'analytics_{p_property_id}' 
        #      and table_name like 'reported_events_2%'
        #      and table_name != 'reported_events_20230205'
        #      UNION ALL
        #      select table_name,'transactions'
        #      from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
        #      where table_schema = 'analytics_{p_property_id}' 
        #      and table_name like 'reported_transactions_2%'
        #      and table_name != 'reported_transactions_20230205'
        #      UNION ALL
        #      select table_name,'acquisition_sessions'
        #      from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
        #      where table_schema = 'analytics_{p_property_id}' 
        #      and table_name like 'reported_acquisition_sessions_2%'
        #      and table_name != 'reported_acquisition_sessions_20230205'
        #      UNION ALL
        #      select table_name,'transaction_qty'
        #      from `ga360-connection-267115.analytics_{p_property_id}.INFORMATION_SCHEMA.TABLES`
        #      where table_schema = 'analytics_{p_property_id}' 
        #      and table_name like 'reported_transaction_qty_2%'
        #      and table_name!= 'reported_transaction_qty_20230205'"""     
           

    df_info_schema = spark.read.format("bigquery")\
        .option("credentials",ga4_credentials)\
        .option("parentProject",'ga360-connection-267115')\
        .load(query)
   
   
             

    #display(df_info_schema)   
    for j in df_info_schema.collect():
        property_date_tup_list.append((j.table_name,p_property_id,j.tab_type))
    return  property_date_tup_list    


In [ ]:
# ga4_or_gau = 'GA4'
# lineage_source = 'GA4'
# p_property_id = 309591384 
# table = 'ga360-connection-267115.analytics_309591384.reported_events_20230205'
# brand = property_brand[p_property_id].split(' ')[0]
# brand_country = property_brand[p_property_id].split(' ')[-1]
# df = spark.read.format("bigquery") \
#     .option("parentProject",project)\
#     .option("table", table) \
#     .option("credentials",ga4_credentials) \
#     .load()

# # df = df.withColumn('property_brand',lit(brand))\
# #            .withColumn('brand_country',lit(brand_country))\
# #            .withColumn('lineage_source',lit(lineage_source))

# # df.repartition('date','property_brand','brand_country')\
# #     .write.format("delta")\
# #     .mode("append")\
# #     .option("path",f"{raw_adls_path}{ga4_or_gau}/events_feb05_nov08/events/")\
# #     .option("mergeSchema", "true")\
# #     .partitionBy('date','property_brand','brand_country')\
# #     .save()
# display(df.count())

In [ ]:

ga4_or_gau = 'GA4'

lineage_source = 'GA4'

def ingest_bq_data(p_table_name,p_property_id,p_tab_type):
  try:
    print('p_table_name::',p_table_name,'p_property_id::',p_property_id)

    brand = property_brand[p_property_id].split(' ')[0]
    brand_country = property_brand[p_property_id].split(' ')[-1]
    #print("part_name::",part_name,"brand::",brand)
    table = f'{project}.analytics_{p_property_id}.{p_table_name}'
    df = spark.read.format("bigquery") \
    .option("parentProject",project)\
    .option("table", table) \
    .option("credentials",ga4_credentials) \
    .load()
    #.where("event_name IN ('view_item', 'add_to_cart') ")
    df = df.withColumn('property_brand',lit(brand))\
           .withColumn('brand_country',lit(brand_country))\
           .withColumn('lineage_source',lit(lineage_source))
    #df_all_ingest.append(df)
    df.repartition('date','property_brand','brand_country')\
    .write.format("delta")\
    .mode("append")\
    .option("path",f"{raw_adls_path}{ga4_or_gau}/events_feb05_nov08/{p_tab_type}/")\
    .option("mergeSchema", "true")\
    .partitionBy('date','property_brand','brand_country')\
    .save()
  except Exception as e:
    print('Other exception::','p_table_name::',p_table_name,'p_property_id::',p_property_id,'::',str(e)) 
    raise 


# MAIN Call - get data from BQ and write to ADLS

In [ ]:
def load_all_data(p_tab_type):
  for i,j in property_brand.items():
    print('processing:: ',j )
    dates_list = get_property_partitions(i,p_tab_type)
    table_list = list(set([x for x in dates_list ]))    
    #table_list = list(set(dates_list))
    #print(sorted(table_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
          results = list(executor.map(ingest_bq_data, *zip(*table_list)))
   


## call load_all_data()

In [ ]:
#tab_type = ['events','transactions','acquisition_sessions','transaction_qty']
#tab_type = ['all']
load_all_data('all')

In [ ]:
print(property_brand.keys())

# Validate the counts

In [ ]:
%%sql
-- update delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transactions/`
-- set date = date_format(to_date(date, 'yyyyMMdd'),'yyyy-MM-dd') 
-- where date = '20230205'


In [ ]:
%%sql
-- select event_date , event_params, event_params.product_price.double_value
-- ,items.item_id
--  ,items.quantity
--  ,items.item_params
--  from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events/` limit 10
--refresh table delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events/`;
-- select date
-- , device_category
-- , stream_name
-- , round(sum(event_count),3) as carts
--  from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
-- --where event_name IN ('add_to_cart')
-- group by date
-- , device_category
-- , stream_name 
-- order by date
-- , device_category
-- , stream_name;

-- select distinct date 
-- , device_category
-- , stream_name
-- , round(sum(event_count),3) as carts
-- from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
-- where 1=1  and date = '2023-02-06' and event_name IN ('add_to_cart')
--     group by date
-- , device_category
-- , stream_name order by date desc

select date_format(to_date(date),'yyyyMMdd')
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
where date = '20230205'

--describe delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`



In [ ]:
%%sql
select date, count(*)
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
group by date order by 1


In [ ]:
%%sql
-- delete from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/acquisition_sessions/`
-- where property_brand='Wolverine'
--     and brand_country = 'Canada' 

In [ ]:
%%sql
select distinct date 
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transaction_qty/`
order by date asc

In [ ]:
%%sql
select tab_type,brand, count(*) cnt from 
(select 'events' as tab_type, date,concat(property_brand,'-',brand_country) as brand
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
union ALL
select 'transactions' as tab, date,concat(property_brand,'-',brand_country) as brand
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transactions/`
union ALL
select 'acquisition_sessions' as tab,date,concat(property_brand,'-',brand_country) as brand
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/acquisition_sessions/`
union all
select 'transaction_qty' as tab,date,concat(property_brand,'-',brand_country) as brand
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transaction_qty/`
)a where date between '2023-02-05' and '2023-11-08'
group by tab_type, brand
order by tab_type


In [ ]:
%%sql
with t_streams as (select 'mobile' as stream_name 
  union all
  select 'tablet'
  union all
  select 'desktop')
, all_dates as (select explode(sequence(to_date('2023-02-05'),to_date('2023-11-08'),interval 1 day)) as date)
all_data as (select * from t_streams,all_dates)

## Validate data

In [ ]:
%%sql
with t_streams as (select 'mobile' as device_category 
  union all
  select 'tablet'
  union all
  select 'desktop')
, all_dates as (select explode(sequence(to_date('2023-02-05'),to_date('2023-11-08'),interval 1 day)) as date)
,all_data as (select * from t_streams,all_dates)
, t1_carts as (select date
, device_category
, stream_name
, round(sum(event_count),3) as carts
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
where event_name IN ('add_to_cart')
group by date
, device_category
, stream_name)
,t2_orders as (select date
, device_category
, stream_name
, round(sum(event_count),3) as orders
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/events/`
where event_name IN ('purchase', 'transaction')
group by date
, device_category
, stream_name )
, t3_sales as (select date
, device_category
, stream_name
, round(sum(total_revenue-tax_amount-shipping_amount),3) as sales
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transactions/`
 group by date
, device_category
, stream_name)
, t4_sessions as
(select  date
, device_category
, stream_name
, sum(sessions) as sessions
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/acquisition_sessions/`
group by date
, device_category
, stream_name)
,t5_qty as (select  date
, device_category
, stream_name
, sum(items_purchased) as qty
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/transaction_qty/`
group by date
, device_category
, stream_name)
select all_data.date
, all_data.device_category
, t1.stream_name
, t1.carts
, t2.orders
, t3.sales
, t4.sessions
, t5.qty
from all_data left join t1_carts t1 on all_data.date = t1.date and all_data.device_category = t1.device_category
 left join t2_orders t2 on t1.date = t2.date and t1.device_category = t2.device_category
and t1.stream_name = t2.stream_name
left join t3_sales t3 on t1.date = t3.date and t1.device_category = t3.device_category
and t1.stream_name = t3.stream_name
left join t4_sessions t4 on t1.date = t4.date and t1.device_category = t4.device_category
and t1.stream_name = t4.stream_name
left join t5_qty t5 on t1.date = t5.date and t1.device_category = t5.device_category
and t1.stream_name = t5.stream_name
where all_data.date = '2023-11-08'

 

In [ ]:
%%sql
select  distinct stream_name
 from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events_feb05_nov08/acquisition_sessions/`
